In [ ]:
# Install the required packages
!pip install ultralytics pillow opencv-python-headless

import cv2
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

# Path to the video file
video_path = "/content/obj detection1.mp4"  # Change this to your video file path

# Load the Haar cascade file for full-body detection
fullbody_cascade_path = 'haarcascade_fullbody.xml'
fullbody_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + fullbody_cascade_path)

# Function to process and annotate video frames
def detect_animals_in_video(video_path):
    # Load YOLOv8 model
    model = YOLO('yolov8s.pt')  # Use 'yolov8s.pt' for the smallest version

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('output_video.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect full bodies using Haar cascade
        fullbodies = fullbody_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=3, minSize=(60, 60), flags=cv2.CASCADE_SCALE_IMAGE)

        # Draw rectangles around detected full bodies
        for (x, y, w, h) in fullbodies:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # Use YOLOv8 model to detect animals
        results = model(frame)
        label_count = {}  # Dictionary to keep track of label counts

        for result in results:
            for box in result.boxes:
                # Access class ID, confidence, and bounding box coordinates
                cls_id = int(box.cls[0])
                label = model.names[cls_id]
                confidence = box.conf[0]

                if label in ['zebra', 'elephant', 'bear', 'giraffe']:  # Add or modify the list as needed
                    x1, y1, x2, y2 = map(int, box.xyxy[0])

                    # Increment the label count or initialize it
                    if label not in label_count:
                        label_count[label] = 1
                    else:
                        label_count[label] += 1

                    # Create the label with index if there are multiple instances
                    label_with_index = f"{label}_{label_count[label]}" if label_count[label] > 1 else f"{label}_1"

                    # Draw rectangle and put text
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                    text = f"{label_with_index} ({confidence:.2f})"
                    # Calculate text background
                    (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 2)
                    cv2.rectangle(frame, (x1, y1 - text_height - 2 * baseline), (x1 + text_width, y1), (255, 255, 255), -1)
                    cv2.putText(frame, text, (x1, y1 - baseline), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)  # Blue color and bold text

        # Write the frame to the output video file
        out.write(frame)

    # Release video objects
    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Display the output video
    display_output_video('output_video.mp4')

# Function to display the output video
def display_output_video(output_video_path):
    cap = cv2.VideoCapture(output_video_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Display the frame using cv2_imshow
        cv2_imshow(frame)

    cap.release()
    cv2.destroyAllWindows()

# Example usage
detect_animals_in_video(video_path)
